# Building

If you have already built the executable as `./build/sdf-weighted-delaunay` feel free to skip this part.

Otherwise, set `build=True` to build. 

In [ ]:
build=False

import os
if build:
    if not os.path.exists("./build"):
        os.mkdir("build")
    !cmake -B build -DCGAL_DIR=./cgal 
    !cmake --build build/ --parallel
    
if os.path.exists("./build/sdf-weighted-delaunay"): 
    print("Executable Available")
else:
    print("Error: Executable not available")

# SDF sampling
The re-sampling requires a sdf function that can be evaluated. This example uses a ground truth mesh and the `signed_distance` function provided by the `gpytoolbox` package.

In [ ]:
import os
# Download the Armadillo from github if it is not available already
mesh_path = "armadillo.obj"
if not os.path.exists(mesh_path) and mesh_path == "armadillo.obj":
    import wget
    url = 'https://raw.githubusercontent.com/alecjacobson/common-3d-test-models/refs/heads/master/data/armadillo.obj'
    filename = wget.download(url)
    print()
    print(filename)
else:
    print("Found .obj file")
    
import numpy as np

def sample_positions(n, random_sampling, sdf):
    if random_sampling:
        U = (np.random.rand(n*n*n, 3)-0.5)*2
    else:
        gx, gy, gz = np.meshgrid(np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1))
        U = np.vstack((gx.flatten(), gy.flatten(), gz.flatten())).T
    U_sdfvals = sdf(U)
    return U, U_sdfvals

import gpytoolbox as gpy

# Set up gt
V_gt, F_gt = gpy.read_mesh(mesh_path)
V_gt = gpy.normalize_points(V_gt)

s = 0.9 # 0.75
V_gt *= s/0.5

# Create and abstract SDF function that is the only connection to the shape
sdf = lambda x: gpy.signed_distance(x, V_gt, F_gt)[0]

# Contouring:

In [ ]:
# number of samples per dimensions
n = 20

# Sample SDF values on a regular grid
U, U_sdfvals = sample_positions(n,False,sdf)

gridpath = "./cgal/tmp.csv"
np.savetxt(gridpath, np.concatenate([U,U_sdfvals[:,None]],axis=1), delimiter=",")

!{cgalbinpath} {gridpath}
cgal_pwns = np.genfromtxt('./pwn.csv', delimiter=',')